In [4]:
import pandas as pd

In [5]:
#! pip install folium

In [6]:
import folium as flm
import os 
import json

In [7]:
player_df = pd.read_csv('Players.csv')
us_states = pd.read_json('states_hash.json') # загружаем файлы для построения карт
state_geo = os.path.join('us-states.json')  # загружаем файлы для построения карт

Посмотрим данные в файле, чтобы понять, что интересное можно построить на графиках. Заодно удалим дубли, если есть и null.

In [8]:
player_df.head()
#player_df.info()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [11]:
player_df.dropna(inplace = True)
player_df.drop_duplicates()
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3189 entries, 1 to 3919
Data columns (total 8 columns):
Unnamed: 0     3189 non-null int64
Player         3189 non-null object
height         3189 non-null float64
weight         3189 non-null float64
collage        3189 non-null object
born           3189 non-null float64
birth_city     3189 non-null object
birth_state    3189 non-null object
dtypes: float64(3), int64(1), object(4)
memory usage: 224.2+ KB


Посмотрев на данные, решаю построить карту США, на которой распределены места рождения игроков в бейсбол. 
Может быть это даст что-то интересное. Для построения использую категориальные данные birth_state, player, obect Для этого загрузили несколько файлов и соединили с исходной таблицей, 
тк для folium нужны аббревиатуры штатов США.

In [23]:
player_states = player_df.merge(us_states, how = 'inner', left_on = 'birth_state', right_on = 'name')
player_states.head()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state,abbreviation,name
0,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana,IN,Indiana
1,9,Vince Boryla,196.0,95.0,University of Denver,1927.0,East Chicago,Indiana,IN,Indiana
2,48,Bob Evans,188.0,79.0,Butler University,1925.0,Indianapolis,Indiana,IN,Indiana
3,100,Leo Klier,188.0,77.0,University of Notre Dame,1923.0,Washington,Indiana,IN,Indiana
4,112,John Logan,188.0,79.0,Indiana University,1921.0,Richmond,Indiana,IN,Indiana


Распределение места рождения игроков по штатам изобразим в процентах, заодно проверим, что вычисления правильные.

In [37]:
states = (player_states.groupby('abbreviation')['Player'].count()*100/player_states.Player.count())\
                                  .to_frame().reset_index()
states.Player.sum()     # проверка правильности рассчета; все ок
states.head(3)

,abbreviation,Player
0,AK,0.033784
1,AL,2.500000
2,AR,1.587838


Инициализируем карту, строим график.

In [38]:
m = flm.Map(location=[37, -102], zoom_start=5)

In [41]:
m.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=states,
 columns=['abbreviation', 'Player'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='BIRTH STATE, USA PLAYERS PERSANTAGE, %'
)
flm.LayerControl().add_to(m)

In [66]:
m

Видим достаточно интересное распределение по штатам. Скорее всего, в восточной части США, а также в Калифорнии
более развита школа бейсбола по сравнению с другими штатами.